## Análsis de la Retroalimentación de Posadas y Gentera GPTW México
## Basado en muestra de datos de Net Promoter Score de ambos hoteles

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
print(os.listdir("../input"))

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.lang.es import Spanish
!python -m spacy download es_core_news_sm

#Pandas display width
pd.set_option('max_colwidth',100) #max width df display

['gptw07', 'gptw08']


    100% |████████████████████████████████| 36.7MB 91.3MB/s ta 0:00:01
  Running setup.py install for es-core-news-sm ... done
You are using pip version 10.0.1, however version 20.3b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /opt/conda/lib/python3.6/site-packages/es_core_news_sm -->
    /opt/conda/lib/python3.6/site-packages/spacy/data/es_core_news_sm

    You can now load the model via spacy.load('es_core_news_sm')



In [2]:
# Data load in csv format
wines = pd.read_csv('../input/gptw08/dbgptwliderplus.csv')

wines.head(5)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
0,Grupo Posadas,4094711993,7,"Es una buena empresa para trabajar ofrecen buenas prestaciones, pero es muy matado el trabajo; t...",A veces,"Más motivación a personal, si el personal está bien y motivado puede hacer mejor y bien su trabajo",Responsable,Buen humor y es tolerante
1,Grupo Posadas,4038461966,10,Poque me gusta mi trabajo,Porlosebfoquenosfestejan,Porquemegustamuchoelhofel,NaN,Estaré pasillos
2,Grupo Posadas,4022001994,10,"Porque es una buena empresa, hay un ambiente laboral agradable,","Buen ambiente de trabajo, beneficios",Prestaciones,"Entusiasta, honesto, retroalimentación",Organizado
3,Grupo Posadas,4032331988,10,Tiene un aviente muy bueno para trabajar,Hay buena relación en el trabajo en trabajo en ejido,El comedor y el horario del trasporte,Te escucha te da consejos no grita,Puede dar oportunidades de como ayer un trabajo de excelente
4,Grupo Posadas,204361993,10,POR EL BUEN AMBIENTE DE TRABAJO. EN ESTA EMPRESA APRENDES MUCHÍSIMAS COSAS,ES UNA ESCUELA PARA APRENDER MUCHAS COSAS,PERSONAL CAPACITADO,SABER DIRIGIR EL DEPARTAMENTO,HACER MAS PROYECTOS DE TRABAJO


In [3]:
wines.shape

(28409, 8)


## Seccción para hacer los queries correspondientess, se requiere la selección de los valores de los campos de interés. 
## El diseño utiliza "wines" como dataframe principal, utilizo "newdf" como auxiliar.  
##  Validando la extracción correcta de la información, str.len para determinar número mínimo de carcteres

In [4]:
#selecting subsets of the dataframe, note the string length condition
#you have to select socre organization and ALSO the name of the field (column) to extract
newdf=wines[(wines['nps_score']>8)  & (wines['org']=="Gentera") & (wines['lider_minus'].str.len()>20)]
newdf.head(6)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
15128,Gentera,175811984,10,por que es una empresa donde la persona es el centro y sobre todo nos llevan a vivir los valores...,"el trato, a pesar de los puestos nos tratamos todos se igualmanera, siempre cuidadon las formas ...",los horarios,sencillez,"aprender a escuchar, involucramiento con sus colaboradores"
15129,Gentera,834331992,10,Porq es una empresa muy comprometida con su gente,El trato a la persona,La empresa está muy estructurada,Su conocimiento,Conocer más nuestra persona
15130,Gentera,526311989,9,HAY OPORTUNIDAD DE CRECIMIENTO,NOS GUIA A TRABAJAR CON VALORES\nNOS DA OPORTUNIDAD DE ESTAR CON NUESTRAS FAMILIAS,QUE NOS DIERAN UN MEDIO DE TRASPORTE SEGURO,RESPONSABLE Y HONESTO,COMUNICACION Y TRABAJO EN EQUIPO
15131,Gentera,840831981,10,Prioriza el valor de la persona y la familia,Prioriza el valor de la persona,No,Conoce los procesos,"Manejó del estrés , su forma de tratar a las personas es muy explosiva"
15132,Gentera,908521986,10,Es una buena empresa,El trabajo en equipo,Todo muy bien,Dinámico activo buen ejemplo a seguir,La capacitación constante
15137,Gentera,861841986,10,Es una buena empresa,El ambiente laboral,Ninguno,El conocimiento,Recordar cuando le pides apoyo de un encargo


In [5]:
newdf.shape

(5202, 8)

## Registra el numero de temas a identificar en la siguiente línea

In [6]:
#Aqui se define el numero de temas a buscar y el nombre del campo del data frame que contiene la informacion a analizar
total_topics=4
field_source='lider_minus'

In [7]:
# Reseting the index of the dataframe ****  *** very important resetting data frame index
newdf=newdf.reset_index(drop=True)
newdf.head(5)

,org,id,nps_score,nps_why,algo_especial_emp,algo_cambiar_emp,lider_plus,lider_minus
0,Gentera,175811984,10,por que es una empresa donde la persona es el centro y sobre todo nos llevan a vivir los valores...,"el trato, a pesar de los puestos nos tratamos todos se igualmanera, siempre cuidadon las formas ...",los horarios,sencillez,"aprender a escuchar, involucramiento con sus colaboradores"
1,Gentera,834331992,10,Porq es una empresa muy comprometida con su gente,El trato a la persona,La empresa está muy estructurada,Su conocimiento,Conocer más nuestra persona
2,Gentera,526311989,9,HAY OPORTUNIDAD DE CRECIMIENTO,NOS GUIA A TRABAJAR CON VALORES\nNOS DA OPORTUNIDAD DE ESTAR CON NUESTRAS FAMILIAS,QUE NOS DIERAN UN MEDIO DE TRASPORTE SEGURO,RESPONSABLE Y HONESTO,COMUNICACION Y TRABAJO EN EQUIPO
3,Gentera,840831981,10,Prioriza el valor de la persona y la familia,Prioriza el valor de la persona,No,Conoce los procesos,"Manejó del estrés , su forma de tratar a las personas es muy explosiva"
4,Gentera,908521986,10,Es una buena empresa,El trabajo en equipo,Todo muy bien,Dinámico activo buen ejemplo a seguir,La capacitación constante


In [8]:
newdf.shape

(5202, 8)

In [9]:
# To review element 
newdf[field_source][4]
#print(newdf.count)

'La capacitación constante '

In [10]:
#Updating the wine df with the processed query
wines=newdf
wines.shape

(5202, 8)

In [11]:
#seleccionando campo fuente del dataframe 
wines[field_source][4]

'La capacitación constante '

In [12]:
# Creando un objeto tipo Spacy
# En este caso el objeto es el modelo en Español
nlp = spacy.load('es_core_news_sm')

In [13]:
#Verficando que se esta procesando el registro correcto
doc = nlp(wines[field_source][4])
spacy.displacy.render(doc, style='ent',jupyter=True)

## Econtrando la raíz de los términos (Lemmatization)
## Creación de lista de puntuación y palabras comunes que no añaden mucha informaciónCY


In [14]:
#definicion de puntuacion y STOP WORDS
punctuations = string.punctuation
stopwords = list(STOP_WORDS)
stopwords.append('empresa') # added 'empresa' as a Stop Word

In [15]:
#Quitar signo de comentarios para revisar listas de stopwords y punctuations
#stopwords
#punctuations

In [16]:
review = str(" ".join([i.lemma_ for i in doc]))

In [17]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

## Detectando los componentes dentro de las frases ###



In [18]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

La => DET
capacitación => NOUN
constante => ADJ


In [19]:
# Parser para las frases escritas Añadi len word >4 to filter out short words with no meaning in this exercise
parser = Spanish()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lower_ for word in mytokens ] #*either this line or the next one
    #mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ] #***this section keeps only the lemma removed
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations and len(word)>4] #Eliminating short words
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [20]:
# utilizando la variable field_source como la columna a analizar de la base de datos
spacy_tokenizer(wines[field_source][4])

'capacitación constante'

In [21]:
spacy_tokenizer('realmente no lo sé xq sólo se dedica a gritarnos  no tiene manera d decir las cosas sin saltarse')

'realmente dedica gritarnos saltarse'

In [22]:
spacy_tokenizer('pues creo q ninguna porque nos trata como personas inutiles aparte es muy alzado hasta vemos')

'personas inutiles aparte alzado vemos'

In [23]:
# Applies the parser and filters information to get a processed data frame
# Important selection on the column of the wines field_source
tqdm.pandas()
wines["processed_source"] = wines[field_source].progress_apply(spacy_tokenizer)

100%|██████████| 5202/5202 [00:02<00:00, 1796.34it/s]


In [24]:
wines["processed_source"][0:4]

0    aprender escuchar involucramiento colaboradores
1                                            persona
2                                comunicacion equipo
3      manejó estrés forma tratar personas explosiva
Name: processed_source, dtype: object

## Encontrando los temas principales entre todos los documentos

## Creación de un vector de términos


## Aquí es donde se hace el fit a la vectorizacion de la data!

In [25]:
# Creating a vectorizer version without the stop words option
vectorizer = CountVectorizer( lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(wines["processed_source"]) #procesando processed source
#type(data_vectorized)
data_vectorized.shape

(5202, 3985)

In [26]:
NUM_TOPICS = total_topics #parametro desde el principio

In [27]:
# Latent Dirichlet Allocation Model ADDED random_state = 42 to reproduce results
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=20, learning_method='online',verbose=True,random_state=42)
data_lda = lda.fit_transform(data_vectorized)
lda.components_.shape
#type(data_lda)
#print(data_lda.shape)
#type(data_lda)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


(4, 3985)

In [28]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [29]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [30]:
# function printing keywords for each topic       *** You may select top terms ***
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):    #idx and topic become a vector for each iteration through the enumerate command
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i])     #very cleverly algins the names in vectorizer with the topics in model components
                        for i in topic.argsort()[:-top_n - 1:-1]]) # i here is the index orders in descinding mode keep the first 10 value stop value is -11 to 

In [31]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
['falta', 'equipo', 'organizaci', 'comunicaci', 'ocasiones', 'confianza', 'puesto', 'caracter', 'actividades', 'tiempos']
Topic 1:
['escuchar', 'colaboradores', 'trato', 'personas', 'seguimiento', 'der', 'apoyo', 'colaborador', 'cargo', 'hablar']
Topic 2:
['persona', 'personal', 'oportunidad', 'apoyo', 'forma', 'mejorar', 'gente', 'car', 'cter', 'campo']
Topic 3:
['equipo', 'organizacion', 'lider', 'clientes', 'paciencia', 'decisiones', 'manejo', 'conocimiento', 'aprender', 'tomar']


## Habilitar para análsis NMF

In [32]:
# Keywords for topics clustered by Latent Semantic Indexing
#print("NMF Model:")
#selected_topics(nmf, vectorizer)

## Habilitar para análsis LSI

In [33]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
#print("LSI Model:")
#selected_topics(lsi, vectorizer)

## Identificando documentos correspondientes a cada tema

In [34]:
# function printing documents belonging to each topic  *** You may select # of top documents ***
def selected_documents(model,wines2,top_n=30):
    numtemas, tempvar = model.components_.shape
    print("Printing Document belonging to each topic")
    for idx in range(numtemas): #idx and topic become a vector for each iteration through each topic 
        print(" ")
        print("Topic %d:" % (idx))
        print(" ")
        ptop_indices=np.argsort(doc_topic[:,idx])[:-top_n-1:-1] #sorting the indices by the offset column 1 in reverse order, top indices gives the order document
        print(wines2[ptop_indices])
        if (idx==0):
            dfcsvtmp=wines2[ptop_indices].to_frame()  #note the use to frame method as the results were just a series
            dfcsvtmp.insert(1,"Tema",idx) #during the first iteration add the column named tema with value idx zero in first iteration
            print(idx)
        else:
            dfcsvtmp2=wines2[ptop_indices].to_frame() #note the use to frame method as the results were just a series
            dfcsvtmp2.insert(1,"Tema",idx) # use a tmp dataframe to add the column named tema and fill it with the current idx iteration
            dfcsvtmp=dfcsvtmp.append(dfcsvtmp2) # append to previous data frame including the tema column
    return dfcsvtmp #returning the data frame including all comments and temas
        #print (doc_topic[ptop_indices]) #Uncomment to see probabilities by topic

In [35]:
print(lda.components_.shape)

(4, 3985)


In [36]:
# for each document calculate the probability for each topic in column form *** VERY IMPORTANT ***
doc_topic=lda.transform(data_vectorized) 
doc_probs=pd.DataFrame(doc_topic)
#Top documents by topic
# You can change to the label "processed_source" below to see the parsed sentences in the following line
dfcsv=selected_documents(lda,wines[field_source].str.lower()) #converting results to lower case

Printing Document belonging to each topic
 
Topic 0:
 
611     cambia constantemente de estrategias sin dar un tiempo determinado para probar funcionen, a vece...
240     le falta trabajar en equipo\nle falta empatia\nevitar las prefetencias\nno conoce la formacion p...
295     planificar mejores estrategias.  saber realmente los problemas que hay en cada cuenta el porcque...
367     falta de liderazgo\ntrabajo en equipo\nacuerdos con terceros sin respaldar al.comaborador\nfalta...
335     implementación de nuevas estrategias de venta, cambiar las estrategias para alcanzar los objetiv...
1135    es muy obsesivo con tener todo perfecto, y esto hace que trabajemos con bitácoras tras bitácoras...
541     no se explicarlo porque a veces siento que le da mucha flohera hacer las cosas pero al fibal de ...
166          liderazgo y trabajo en equipo, sabe guiar y orientar, fortalece las actividades mas complejas.
3143    es importante integrar al equipo se pueden realizar actividades que fomen

In [37]:
dfcsv


,lider_minus,Tema
611,"cambia constantemente de estrategias sin dar un tiempo determinado para probar funcionen, a vece...",0
240,le falta trabajar en equipo\nle falta empatia\nevitar las prefetencias\nno conoce la formacion p...,0
295,planificar mejores estrategias. saber realmente los problemas que hay en cada cuenta el porcque...,0
367,falta de liderazgo\ntrabajo en equipo\nacuerdos con terceros sin respaldar al.comaborador\nfalta...,0
335,"implementación de nuevas estrategias de venta, cambiar las estrategias para alcanzar los objetiv...",0
1135,"es muy obsesivo con tener todo perfecto, y esto hace que trabajemos con bitácoras tras bitácoras...",0
541,no se explicarlo porque a veces siento que le da mucha flohera hacer las cosas pero al fibal de ...,0
166,"liderazgo y trabajo en equipo, sabe guiar y orientar, fortalece las actividades mas complejas.",0
3143,es importante integrar al equipo se pueden realizar actividades que fomenten la confianza y trab...,0
4909,falta de organización en eventos extras del ambiente laboral,0


In [38]:
doc_probs.iloc[4059,0:4]

0    0.025023
1    0.924933
2    0.025021
3    0.025023
Name: 4059, dtype: float64

In [39]:
tableau=pd.concat([dfcsv, doc_probs], axis=1, join='inner') #inner joing by index of top n doc with idx and probs ****


In [40]:
tableau.to_csv('topicdb.csv')

In [41]:
doc_probs

,0,1,2,3
0,0.050002,0.450182,0.050002,0.449814
1,0.125001,0.125001,0.624996,0.125001
2,0.743227,0.083334,0.083334,0.090104
3,0.035721,0.036857,0.891662,0.035760
4,0.083342,0.416829,0.083341,0.416489
5,0.050036,0.848132,0.051795,0.050037
6,0.313481,0.062517,0.311885,0.312117
7,0.062514,0.062516,0.062513,0.812457
8,0.083406,0.083414,0.749772,0.083408
9,0.062540,0.811137,0.063782,0.062541


In [42]:
type(wines)

pandas.core.frame.DataFrame

## Seccion de prueba para una frase

In [43]:
# Example of transformation of an individual sentence with its probabilities for each topic
text = spacy_tokenizer("Conoce mucho es perservarnte y simpre predica con el ejemplo muy inteligente")
x = lda.transform(vectorizer.transform([text]))[0]
print('Probabilities by topic (count start with zero): ',x)
print()
print("El tema con mayor probabilidad usando modelo LDA es el tema #",np.argmax(x))

Probabilities by topic (count start with zero):  [0.45364688 0.44625191 0.05004968 0.05005153]

El tema con mayor probabilidad usando modelo LDA es el tema # 0


## Visualizando los Resultado de LDA con la utilería pyLDAvis

In [44]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      -78.671196 -396.283112       1        1  27.474965
0      -33.778664   -7.443441       2        1  25.272522
3      138.194778 -224.309540       3        1  24.350190
1     -250.644653 -179.417053       4        1  22.902322, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
1777  Default  231.000000          falta  231.000000  30.0000  30.0000
1541  Default  379.000000         equipo  379.000000  29.0000  29.0000
1578  Default  170.000000       escuchar  170.000000  28.0000  28.0000
668   Default  144.000000  colaboradores  144.000000  27.0000  27.0000
2765  Default  106.000000     organizaci  106.000000  26.0000  26.0000
2897  Default  114.000000        persona  114.000000  25.0000  25.0000
768   Default  106.000000     comunicaci  106.000000  24.0000  24.0000
2898  Default  107.000000       personal  107.000000  23.0000  23.0000
2678  Default   89.000000      ocasiones   89.000000  22.0000  22.0000
3869  Default   79.000000          trato   79.000000  21.0000  21.0000
805   Default   83.000000      confianza   83.000000  20.0000  20.0000
2768  Default   80.000000   organizacion   80.000000  19.0000  19.0000
2737  Default  105.000000    oportunidad  105.000000  18.0000  18.0000
1843  Default   84.000000          forma   84.000000  17.0000  17.0000
3176  Default   76.000000         puesto   76.000000  16.0000  16.0000
3511  Default   69.000000    seguimiento   69.000000  15.0000  15.0000
2457  Default   79.000000        mejorar   79.000000  14.0000  14.0000
1909  Default   79.000000          gente   79.000000  13.0000  13.0000
569   Default   72.000000       caracter   72.000000  12.0000  12.0000
1014  Default   76.000000           cter   76.000000  11.0000  11.0000
566   Default   76.000000            car   76.000000  10.0000  10.0000
91    Default   71.000000    actividades   71.000000   9.0000   9.0000
543   Default   74.000000          campo   74.000000   8.0000   8.0000
2314  Default   65.000000          lider   65.000000   7.0000   7.0000
304   Default  128.000000          apoyo  128.000000   6.0000   6.0000
2904  Default  107.000000       personas  107.000000   5.0000   5.0000
3184  Default   69.000000    puntualidad   69.000000   4.0000   4.0000
646   Default   61.000000       clientes   61.000000   3.0000   3.0000
3784  Default   60.000000        tiempos   60.000000   2.0000   2.0000
3262  Default   55.000000           reas   55.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
2900   Topic4   30.548758     personales   31.176244   1.4536  -4.8679
277    Topic4   30.230772       apertura   30.852407   1.4536  -4.8783
2591   Topic4   29.588951    necesidades   30.220070   1.4528  -4.8998
3016   Topic4   28.818608   preferencias   29.442827   1.4525  -4.9262
820    Topic4   28.347390     congruente   28.969886   1.4522  -4.9427
645    Topic4   28.473225        cliente   29.098677   1.4522  -4.9382
870    Topic4   22.899483      constante   23.520639   1.4472  -5.1561
913    Topic4   21.767887      controlar   22.392546   1.4456  -5.2067
1921   Topic4   20.049453          gesti   20.673955   1.4433  -5.2890
446    Topic4   18.907659          ayuda   19.540928   1.4410  -5.3476
3745   Topic4   18.364301   temperamento   18.983497   1.4408  -5.3768
539    Topic4   17.730515        cambios   18.352412   1.4395  -5.4119
2348   Topic4   19.233240          llega   19.910463   1.4393  -5.3305
2558   Topic4   17.061460        motivar   17.690064   1.4378  -5.4504
889    Topic4   16.090897      contestar   16.701743   1.4367  -5.5089
2012   Topic4   15.328974       horarios   15.958850   1.4337  -5.5574
717    Topic4   15.203762      compartir   15.833789   1.4333  -5.5656
3108   Topic4   14.855974       producto   15.485454   1.43

In [45]:
#print(dash.topic_coordinates)
coordinates=pd.DataFrame(dash.topic_coordinates)
coordinates.head(10)
coordinates.to_csv('coordinates2.csv')

In [46]:
#print(dash.topic_info)
topic_info=pd.DataFrame(dash.topic_info)
topic_info.head(20)
topic_info.to_csv('topic_info2.csv')

In [47]:
#print(dash.token_table)
tok_table=pd.DataFrame(dash.token_table)
tok_table.head(20)
tok_table.to_csv('tok_table2.csv')

In [48]:
#pyLDAvis.enable_notebook()
#dash = pyLDAvis.sklearn.prepare(nmf, data_vectorized, vectorizer, mds='tsne')
#dash

## Cómo interpretar ésta gráfica?
1. Los temas están a la izquierda y sus palabras respectivas a la derecha.
2. Los temas más grandes con más frecuents y entre más cercanos más parecidos son.
3. La selección de las palabras está basada en su capacidad de diferenciación y frecuencia.

**Selecciona el tema para ver sus palabras correspondientes.**

## Biagram spaCy tokenizer para la identificación de temas

In [49]:
def spacy_bigram_tokenizer(phrase):
    doc = parser(phrase) # create spacy object
    token_not_noun = []
    notnoun_noun_list = []
    noun = ""

    for item in doc:
        if item.pos_ != "NOUN": # separate nouns and not nouns
            token_not_noun.append(item.text)
        if item.pos_ == "NOUN":
            noun = item.text
        
        for notnoun in token_not_noun:
            notnoun_noun_list.append(notnoun + " " + noun)

    return " ".join([i for i in notnoun_noun_list])

In [50]:
bivectorizer = CountVectorizer(min_df=5, max_df=0.9, lowercase=True, ngram_range=(1,2))
bigram_vectorized = bivectorizer.fit_transform(wines["processed_source"])

## LDA para información procesada con Biagram

In [51]:
bi_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_bi_lda = bi_lda.fit_transform(bigram_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


### Temas pare el modelo de bigram 

In [52]:
print("Bi-LDA Model:")
selected_topics(bi_lda, bivectorizer)

Bi-LDA Model:
Topic 0:
['apoyo', 'mejorar', 'seguimiento', 'clientes', 'personal', 'laboral', 'crecimiento', 'crecer', 'colaborador', 'promotores']
Topic 1:
['escuchar', 'persona', 'personas', 'colaboradores', 'organización', 'caracter', 'escucha', 'lider', 'tiempos', 'trato']
Topic 2:
['falta', 'ocasiones', 'organizacion', 'puesto', 'campo', 'decisiones', 'gente', 'errores', 'actividades', 'experiencia']
Topic 3:
['equipo', 'comunicación', 'oportunidad', 'forma', 'confianza', 'puntualidad', 'carácter', 'áreas', 'humildad', 'manejo']


In [53]:
bi_dash = pyLDAvis.sklearn.prepare(bi_lda, bigram_vectorized, bivectorizer, mds='tsne')
bi_dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3       86.109734  -58.441341       1        1  27.070087
1     -112.857552  -48.967510       2        1  25.689715
0       76.634979 -257.408569       3        1  23.867555
2     -122.332306 -247.934799       4        1  23.372643, topic_info=     Category        Freq   ...     loglift  logprob
term                        ...                     
265   Default  440.000000   ...     30.0000  30.0000
320   Default  275.000000   ...     29.0000  29.0000
279   Default  199.000000   ...     28.0000  28.0000
43    Default  149.000000   ...     27.0000  27.0000
541   Default  132.000000   ...     26.0000  26.0000
545   Default  123.000000   ...     25.0000  25.0000
518   Default  122.000000   ...     24.0000  24.0000
132   Default  122.000000   ...     23.0000  23.0000
497   Default  107.000000   ...     22.0000  22.0000
510   Default  119.000000   ...     21.0000  21.0000
516   Default   98.000000   ...     20.0000  20.0000
590   Default   92.000000   ...     19.0000  19.0000
459   Default   91.000000   ...     18.0000  18.0000
115   Default  166.000000   ...     17.0000  17.0000
89    Default   88.000000   ...     16.0000  16.0000
344   Default   96.000000   ...     15.0000  15.0000
137   Default   95.000000   ...     14.0000  14.0000
644   Default   79.000000   ...     13.0000  13.0000
199   Default   77.000000   ...     12.0000  12.0000
96    Default   82.000000   ...     11.0000  11.0000
277   Default   78.000000   ...     10.0000  10.0000
432   Default   77.000000   ...      9.0000   9.0000
109   Default   72.000000   ...      8.0000   8.0000
597   Default   79.000000   ...      7.0000   7.0000
689   Default   69.000000   ...      6.0000   6.0000
428   Default   59.000000   ...      5.0000   5.0000
178   Default   58.000000   ...      4.0000   4.0000
360   Default   93.000000   ...      3.0000   3.0000
718   Default   61.000000   ...      2.0000   2.0000
177   Default   54.000000   ...      1.0000   1.0000
...       ...         ...   ...         ...      ...
89     Topic4   87.630286   ...      1.4454  -3.5486
199    Topic4   77.063867   ...      1.4443  -3.6771
276    Topic4   50.330652   ...      1.4392  -4.1031
310    Topic4   48.208540   ...      1.4389  -4.1462
693    Topic4   46.308084   ...      1.4383  -4.1864
363    Topic4   45.388096   ...      1.4379  -4.2065
662    Topic4   46.468855   ...      1.4379  -4.1829
349    Topic4   45.600276   ...      1.4377  -4.2018
622    Topic4   42.196038   ...      1.4365  -4.2794
247    Topic4   38.577218   ...      1.4348  -4.3691
544    Topic4   36.428646   ...      1.4340  -4.4264
578    Topic4   36.237213   ...      1.4336  -4.4316
483    Topic4   35.737130   ...      1.4334  -4.4455
576    Topic4   35.553653   ...      1.4332  -4.4507
62     Topic4   34.999202   ...      1.4330  -4.4664
535    Topic4   34.321130   ...      1.4325  -4.4860
628    Topic4   33.920663   ...      1.4320  -4.4977
128    Topic4   30.265113   ...      1.4302  -4.6117
641    Topic4   28.381697   ...      1.4285  -4.6760
438    Topic4   28.575483   ...      1.4284  -4.6692
245    Topic4   27.566583   ...      1.4274  -4.7051
165    Topic4   26.522318   ...      1.4261  -4.7437
122    Topic4   25.257140   ...      1.4255  -4.7926
528    Topic4   22.926543   ...      1.4229  -4.8894
624    Topic4   22.830215   ...      1.4226  -4.8936
688    Topic4   21.737479   ...      1.4212  -4.9427
260    Topic4   21.794458   ...      1.4210  -4.9401
360    Topic4   66.326203   ...      1.1102  -3.8271
18     Topic4   50.165627   ...      0.9348  -4.1064
98     Topic4   25.674623   ...      0.3984  -4.7762

[172 rows x 6 columns], token_table=      Topic      Freq               Term
term                                    
10        1  0.962781       acercamiento
13        2  0.990555     acompañamiento
15        3  0.989137            actitud
18        1  0.403409       

In [54]:
import pandas as pd
master_list = [['cat', 123, 'yellow'], ['dog', 12345, 'green'], ['horse', 123456, 'red']]
df = pd.DataFrame(master_list)

df

,0,1,2
0,cat,123,yellow
1,dog,12345,green
2,horse,123456,red


In [55]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.colheader_justify','light', 'display.width', 2000, 'display.max_colwidth', 500):
    df = df.stack().str.lstrip().unstack()
    df = df.style.set_properties(**{'text-align': 'left'})
df

,0,1,2
0,cat,nan,yellow
1,dog,nan,green
2,horse,nan,red


In [56]:
from collections import Counter
topics = lda.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
    for word, weight in topic:
        out.append([word, i , weight, counter[word]])

df = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])        


AttributeError: 'LatentDirichletAllocation' object has no attribute 'show_topics'

In [ ]:
doc_topic


In [ ]:
dir(LatentDirichletAllocation)

In [ ]:
dir(selected_topics)

In [ ]:
dir(lda)

In [ ]:
selected_topics